<a href="https://colab.research.google.com/github/kumar-abhishek/handson-ml2/blob/master/padded_BachChorales_HandsOnChapter_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
print(X)
print(y)

[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]
[40 50 60 70 80 90]


In [0]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features))) model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
                                                                                   
print(yhat)

In [0]:
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
    return array(X), array(y)
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X)
print(y)
print(type(X))
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(n_features)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
[[[10 15 25]
  [20 25 45]
  [30 35 65]]]
[[40 45 85]]
<class 'numpy.ndarray'>
3


In [0]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu')) 
model.add(Dense(n_features)) 
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=1000, verbose=0)


In [0]:
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[197.93314 223.3317  407.88092]]


**Q10[part-1]. Download the Bach chorales dataset and unzip it. It is composed of 382 chorales composed by Johann Sebastian Bach. Each chorale is 100 to 640 time steps long, and each time step contains 4 integers, where each integer corresponds to a note’s index on a piano (except for the value 0, which means that no note is played). Train a model—recurrent, convolutional, or both—that can predict the next time step (four notes), given a sequence of time steps from a chorale. **bold text**

In [3]:
from google.colab import files
uploaded = files.upload()

In [0]:
uploaded_validation = files.upload()

Saving chorale_229.csv to chorale_229.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import numpy as np

X_train_000 = pd.read_csv('chorale_000.csv')
print(X_train_000.head(10))

X_valid_229 = pd.read_csv('chorale_229.csv')
print(X_valid_229.head(10))

X_train_000 = X_train_000.to_numpy()
X_valid_229 = X_valid_229.to_numpy()

dataX=X_train_000[:-1]
dataY=X_train_000[1:]
dataValidationX=X_valid_229[:-1]
dataValidationY=X_valid_229[1:]

#dataValidationY=[]
#for row in range(len(X_train_000)-1):
#    dataX.append(X_train_000[row])
#    dataY.append(X_train_000[row+1])
print(dataX[0:10])
print(dataY[0:10])
print(type(dataX))

#for row in range(len(X_valid_229)-1):
#     dataValidationX.append(X_valid_229[row])
#     dataValidationY.append(X_valid_229[row+1])
print(dataValidationX[0:5])
print(dataValidationY[0:5])

   note0  note1  note2  note3
0     74     70     65     58
1     74     70     65     58
2     74     70     65     58
3     74     70     65     58
4     75     70     58     55
5     75     70     58     55
6     75     70     60     55
7     75     70     60     55
8     77     69     62     50
9     77     69     62     50
   note0  note1  note2  note3
0     72     67     60     48
1     72     67     60     48
2     72     67     60     48
3     72     67     60     48
4     72     67     64     48
5     72     67     64     48
6     72     67     64     50
7     72     67     64     50
8     72     67     64     52
9     72     67     64     52
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]]
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]
 [77 69 62 50]]
<class 'numpy.ndarray'>
[[72 67 60 48]

In [3]:
"""X = np.reshape(dataX, (len(dataX), 1, 4))
X_train, X_test, y_train,  y_test = train_test_split(X, dataY, test_size=0.33, random_state=42)
print(X_train[0:5])
print(y_train[0:5])
"""
from keras.preprocessing.sequence import pad_sequences
import math
# convert into input/output
i,j=0,0
k,l=0,0
X, y=[], []
validX, validY=[], []
#print(type(dataX))
#print(type(dataX[0]), dataX[0])
while i<len(dataX):
  b=[]
  for item in range(0, i+1):
    b.append(dataX[item])
  X.append(np.array(b))
  i += 1

while j<len(dataY):
  y.append(dataY[j])
  j+= 1

while k<len(dataValidationX):
  b=[]
  for item in range(0, k+1):
    b.append(dataValidationX[item])
  validX.append(np.array(b))
  k += 1


# while k<len(dataValidationX):
#   validX.append(np.array([dataValidationX[k]]))
#   k += 1

while l<len(dataValidationY):
  validY.append(dataValidationY[l])
  l+=1

X=np.array(X)
y=np.array(y)
X=pad_sequences(X, padding='pre')

#print(padded)

print(X[0:2])
print(y[0])
print(X.shape, y.shape)


validX=np.array(validX)
validY=np.array(validY)
validX=pad_sequences(validX, padding='pre')
print(validX[0:2])
print(validY[0])
print(validX.shape, validY.shape)

[[[ 0  0  0  0]
  [ 0  0  0  0]
  [ 0  0  0  0]
  ...
  [ 0  0  0  0]
  [ 0  0  0  0]
  [74 70 65 58]]

 [[ 0  0  0  0]
  [ 0  0  0  0]
  [ 0  0  0  0]
  ...
  [ 0  0  0  0]
  [74 70 65 58]
  [74 70 65 58]]]
[74 70 65 58]
(191, 191, 4) (191, 4)
[[[ 0  0  0  0]
  [ 0  0  0  0]
  [ 0  0  0  0]
  ...
  [ 0  0  0  0]
  [ 0  0  0  0]
  [72 67 60 48]]

 [[ 0  0  0  0]
  [ 0  0  0  0]
  [ 0  0  0  0]
  ...
  [ 0  0  0  0]
  [72 67 60 48]
  [72 67 60 48]]]
[72 67 60 48]
(195, 195, 4) (195, 4)


Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# the dataset knows the number of features, e.g. 2
#n_features = X.shape[2]
n_features = 4
print(n_features)

# choose a number of time steps
n_steps = None

model_lstm = keras.models.Sequential([
    keras.layers.LSTM(100, return_sequences=True, input_shape=[n_steps, n_features]),
    keras.layers.LSTM(100),
    keras.layers.Dense(n_features)
])
model_lstm.compile(optimizer='adam', loss='mse')
# fit model
#model_lstm.fit(X, y, epochs=2, validation_data=(validX, validY))
model_lstm.fit(X, y, epochs=200, validation_data=(validX, validY))

4
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 191 samples, validate on 195 samples
Epoch 1/200
191/191 [==============================] - 6s 32ms/sample - loss: 3944.0617 - val_loss: 3868.4641
Epoch 2/200
191/191 [==============================] - 4s 24ms/sample - loss: 3765.3594 - val_loss: 3708.4227
Epoch 3/200
191/191 [==============================] - 4s 23ms/sample - loss: 3598.7326 - val_loss: 3529.2496
Epoch 4/200
191/191 [==============================] - 4s 23ms/sample - loss: 3427.6683 - val_loss: 3382.4205
Epoch 5/200
191/191 [==============================] - 4s 23ms/sample - loss: 3302.5679 - val_loss: 3278.8151
Epoch 6/200
191/191 [==============================] - 4s 23ms/sample - loss: 3206.6214 - val_loss: 3190.3785
Epoch 7/200
128/191 [===================>..........] - ETA: 1s - loss: 3123.2061

In [1]:
# demonstrate prediction
x_input = X[12]
x_input = x_input.reshape((1, len(x_input), 4))
print(x_input)
yhat = model_lstm.predict(x_input, verbose=0)
print(yhat)
print('expected: ', dataY[12])

NameError: ignored

**Q10[part-2] Then use this model to generate Bach-like music, one note at a time: you can do this by giving the model the start of a chorale and asking it to predict the next time step, then appending these time steps to the input sequence and asking the model for the next note, and so on. Also make sure to check out Google’s Coconet model, which was used for a nice Google doodle about Bach.**

In [129]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
x_input = np.array([dataX[0]])
x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(X):
  # demonstrate prediction
  #x_input = x_input.reshape((1, len(x_input), n_features))
  print('Input: ', x_input)
  print('---------------')
  print('Expected Output: ', y[i])
  print('---------------')
  yhat = model_lstm.predict(x_input, verbose=1)
  print('Predicted Output: ', yhat)
  print("\n\n")
  yhat = yhat.reshape((1, len(yhat), n_features))
  #print(x_input, yhat)
  #print(x_input.shape, yhat.shape)
  x_input = np.concatenate([x_input, yhat], axis=1)
  #print(x_input.shape)
  #print(x_input)
  i += 1
  if i>10:
    break

Input:  [[[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 3ms/sample
Predicted Output:  [[12.871851 11.863461 10.860649  9.285336]]



Input:  [[[74.         70.         65.         58.        ]
  [12.87185097 11.86346054 10.86064911  9.28533554]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicted Output:  [[30.18838  27.720318 25.161957 21.748697]]



Input:  [[[74.         70.         65.         58.        ]
  [12.87185097 11.86346054 10.86064911  9.28533554]
  [30.18837929 27.72031784 25.16195679 21.74869728]]]
---------------
Expected Output:  [75 70 58 55]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicted Output:  [[56.869865 52.05979  47.06205  40.754715]]



Input:  [[[74.         70.         65.         58.        ]
  [12.87185097 11.86346054 10.86064911  9.28533554]
  [30.18837929 27.72031784 25.161

In [130]:
# Using SimpleRNN 
print(n_features)
model_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(100, return_sequences=True, input_shape=[None, n_features]),
    keras.layers.SimpleRNN(100),
    keras.layers.Dense(n_features)
])

model_rnn.compile(loss="mse", optimizer="adam")
print(X.shape, y.shape)
history = model_rnn.fit(X, y, epochs=500,
                    validation_data=(validX, validY))

4
(191, 191, 4) (191, 4)
Train on 191 samples, validate on 195 samples
Epoch 1/500
191/191 [==============================] - 3s 17ms/sample - loss: 3884.6061 - val_loss: 3738.2476
Epoch 2/500
191/191 [==============================] - 1s 4ms/sample - loss: 3597.8243 - val_loss: 3501.6842
Epoch 3/500
191/191 [==============================] - 1s 4ms/sample - loss: 3393.6590 - val_loss: 3344.9362
Epoch 4/500
191/191 [==============================] - 1s 4ms/sample - loss: 3272.6073 - val_loss: 3257.6440
Epoch 5/500
191/191 [==============================] - 1s 4ms/sample - loss: 3191.5464 - val_loss: 3182.6286
Epoch 6/500
191/191 [==============================] - 1s 4ms/sample - loss: 3118.7115 - val_loss: 3110.6376
Epoch 7/500
191/191 [==============================] - 1s 4ms/sample - loss: 3049.0692 - val_loss: 3040.0778
Epoch 8/500
191/191 [==============================] - 1s 4ms/sample - loss: 2978.6175 - val_loss: 2971.6227
Epoch 9/500
191/191 [==============================] - 1

In [133]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
#x_input = X[0]
x_input = np.array([dataX[0]])

x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(X):
  # demonstrate prediction
  #x_input = x_input.reshape((1, len(x_input), n_features))
  print('Input: ', x_input)
  print('---------------')
  print('Expected Output: ', y[i])
  print('---------------')
  yhat = model_rnn.predict(x_input, verbose=1)
  print('Predicted Output: ', yhat)
  print("\n\n")
  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input:  [[[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 3ms/sample
Predicted Output:  [[72.12551  65.19431  58.163357 50.04461 ]]



Input:  [[[74.         70.         65.         58.        ]
  [72.12551117 65.19431305 58.16335678 50.04460907]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicted Output:  [[74.03032  67.032486 59.8005   51.336597]]



Input:  [[[74.         70.         65.         58.        ]
  [72.12551117 65.19431305 58.16335678 50.04460907]
  [74.03031921 67.03248596 59.80049896 51.33659744]]]
---------------
Expected Output:  [75 70 58 55]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicted Output:  [[74.03046  67.032616 59.80062  51.336697]]



Input:  [[[74.         70.         65.         58.        ]
  [72.12551117 65.19431305 58.16335678 50.04460907]
  [74.03031921 67.03248596 59.800

In [132]:

# using GRU
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(n_features)

# choose a number of time steps
n_steps = None

model_gru = keras.models.Sequential([
    keras.layers.GRU(100, return_sequences=True, input_shape=[n_steps, n_features]),
    keras.layers.GRU(100),
    keras.layers.Dense(n_features)
])
model_gru.compile(optimizer='adam', loss='mse')
# fit model
model_gru.fit(X, y, epochs=250, validation_data=(validX, validY))


4
Train on 191 samples, validate on 195 samples
Epoch 1/250
191/191 [==============================] - 7s 38ms/sample - loss: 3954.2694 - val_loss: 3808.7841
Epoch 2/250
191/191 [==============================] - 4s 20ms/sample - loss: 3655.6493 - val_loss: 3534.8152
Epoch 3/250
191/191 [==============================] - 4s 20ms/sample - loss: 3411.2531 - val_loss: 3333.9724
Epoch 4/250
191/191 [==============================] - 4s 20ms/sample - loss: 3238.6062 - val_loss: 3211.2050
Epoch 5/250
191/191 [==============================] - 4s 20ms/sample - loss: 3144.1109 - val_loss: 3131.7926
Epoch 6/250
191/191 [==============================] - 4s 21ms/sample - loss: 3067.1131 - val_loss: 3056.0901
Epoch 7/250
191/191 [==============================] - 4s 21ms/sample - loss: 2992.6149 - val_loss: 2982.3321
Epoch 8/250
191/191 [==============================] - 4s 21ms/sample - loss: 2920.1653 - val_loss: 2910.4434
Epoch 9/250
191/191 [==============================] - 4s 20ms/sample - 

KeyboardInterrupt: ignored

In [0]:
# demonstrate prediction
x_input = X[0:5]
x_input = x_input.reshape((1, len(x_input), n_features))
print(x_input)
yhat = model_gru.predict(x_input, verbose=0)
print(yhat)

[[[74 70 65 58]
  [74 70 65 58]
  [74 70 65 58]
  [74 70 65 58]
  [75 70 58 55]]]
[[73.542854 66.80782  59.76219  51.33617 ]]


In [0]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
#x_input = X[0]
x_input = np.array([dataX[0]])
x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(X):
  # demonstrate prediction
  print('Input: ', x_input)
  print('---------------')
  print('Expected Output: ', y[i])
  print('---------------')
  yhat = model_gru.predict(x_input, verbose=1)
  print('Predicated Output: ', yhat)
  print("\n\n")
  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>10:
    break

Input:  [[[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 5ms/sample
Predicated Output:  [[73.5445   66.80927  59.76339  51.336994]]



Input:  [[[74.         70.         65.         58.        ]
  [73.54450226 66.80927277 59.76338959 51.33699417]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicated Output:  [[73.54287  66.80782  59.762207 51.33618 ]]



Input:  [[[74.         70.         65.         58.        ]
  [73.54450226 66.80927277 59.76338959 51.33699417]
  [73.54286957 66.80782318 59.76220703 51.33618164]]]
---------------
Expected Output:  [75 70 58 55]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicated Output:  [[73.542854 66.807816 59.76219  51.336174]]



Input:  [[[74.         70.         65.         58.        ]
  [73.54450226 66.80927277 59.76338959 51.33699417]
  [73.54286957 66.80782318 59.